<a href="https://colab.research.google.com/github/linyu2703/nhanes_inferential_2021_23/blob/main/inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# importing libraries
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

In [16]:
# loading and reading datasets
demo_src = pd.read_sas('DEMO_L.xpt')
bpxo_src = pd.read_sas('BPXO_L.xpt')
hepb_src = pd.read_sas('HEPB_S_L.xpt')
vid_src = pd.read_sas('VID_L.xpt')
kiq_src = pd.read_sas('KIQ_U_L.xpt')
paq_src = pd.read_sas('PAQ_L.xpt')
whq_src = pd.read_sas('WHQ_L.xpt')

In [17]:
# creating a separate dataframe from the main src dataframe with specific columns
# creating a main dataframe with those dataframes merged by SEQN along with the other specified columns
demo = demo_src[['SEQN','DMDMARTZ','DMDEDUC2','RIDAGEYR']].copy()
bpxo = bpxo_src[['SEQN','BPXOSY3','BPXODI3']].copy()
hepb = hepb_src[['SEQN', 'LBXHBS']].copy()
vid = vid_src[['SEQN', 'LBDVD2LC']].copy()
kiq = kiq_src[['SEQN', 'KIQ022']].copy()
paq = paq_src[['SEQN', 'PAD680']].copy()
whq = whq_src[['SEQN', 'WHD020']].copy()
df = demo.copy()
df = (demo
      .merge(bpxo, on='SEQN', how='left')
      .merge(hepb, on='SEQN', how='left')
      .merge(vid, on='SEQN', how='left')
      .merge(kiq, on='SEQN', how='left')
      .merge(paq, on='SEQN', how='left')
      .merge(whq, on='SEQN', how='left')
      )

In [18]:
# renaming columns
df_cleaned = df[['SEQN','DMDMARTZ','DMDEDUC2','RIDAGEYR','BPXOSY3','BPXODI3','LBXHBS','LBDVD2LC','KIQ022','PAD680','WHD020']].copy().rename(columns={
    'SEQN': 'ID',
    'DMDMARTZ': 'Marital Status',
    'DMDEDUC2': 'Education Level',
    'RIDAGEYR': 'Age',
    'BPXOSY3': 'Systolic BP',
    'BPXODI3': 'Diastolic BP',
    'LBXHBS': 'Hep B',
    'LBDVD2LC': 'Vitamin D',
    'KIQ022': 'Kidney Condition',
    'PAD680': 'Physical Activity',
    'WHD020': 'Weight'
})
# recoding
df_cleaned['Marital Status'] = np.where(df_cleaned['Marital Status'] == 1, 'married',
                                   np.where(df_cleaned['Marital Status'].isin([2,3,77,99]), 'not married', pd.NA))
df_cleaned['Education Level'] = np.where(df_cleaned['Education Level'].isin([4,5]),'bachelors or higher',
                                    np.where(df_cleaned['Education Level'].isin([1,2,3,7,9]),'less than bachelors', pd.NA))
df_cleaned['Age'] = df_cleaned['Age'].mask(df_cleaned['Age'].abs() < 1e-10, np.nan)
df_cleaned['Kidney Condition'] = np.where(df_cleaned['Kidney Condition'] == 1, 'Yes', np.where(df_cleaned['Kidney Condition'] == 2, 'No', pd.NA))
df_cleaned['Physical Activity'] = df_cleaned['Physical Activity'].replace([7777,9999], np.nan)
df_cleaned['Weight'] = df_cleaned['Weight'].replace([7777,9999], np.nan)

df_cleaned['Physical Activity'] = df_cleaned['Physical Activity'].fillna(round(df_cleaned['Physical Activity'].median(), 0))
df_cleaned['Weight'] = df_cleaned['Weight'].fillna(round(df_cleaned['Weight'].mean(), 0))